## Importar todos los modulos necesarios

In [328]:
import pandas as pd
import numpy as np
import uuid
from scipy.stats import truncnorm 

## Random Values Generators

In [332]:
# Funcion normal truncada
def generate_truncated_normal_data(min_val, max_val, std, mean, size):
    a, b = (min_val - mean) / std, (max_val - mean) / std
    data = truncnorm(a, b, loc=mean, scale=std).rvs(size)
    data = np.round(data, 3)
    return data

# Generar una serie de valores aleatorios de tipo categoria con tamanio relativo dependiente del parametro size
def generate_category_random_values(arr, size):
    return pd.Series(np.random.choice(arr, size=size))

# Generar un arreglo de valores aleatorios de tipo unique con tamanio relativo dependiente del parametro size
def generate_unique_random_values(size):
    return [uuid.uuid4().hex[:16] for _ in range(size)]

# Generar un arreglo de valores aleatorios de tipo date con tamanio relativo dependiente del parametro size
def generate_date_random_values(min, max, size):
    date_range = pd.date_range(start=min, end=max, freq='D')
    return np.random.choice(date_range, size)

# Generar un arreglo  de valores aleatorios de tipo foreign con tamanio relativo dependiente del parametro size
def generate_foreign_random_values(serie, size):
    return np.random.choice(serie.drop_duplicates(), size)

# Generar un arreglo  de valores aleatorios de tipo numeric con tamanio relativo dependiente del parametro size
def generate_numeric_random_values(min, max, std, mean, size):
    if std is not None and mean is not None:
        values = generate_truncated_normal_data(min, max, std, mean, size)
    else:
        values = np.random.uniform(min, max, size)
        values = np.round(values, 3)
    return values 

## Random Dataset Simulations Generators

In [446]:
# Generar n cantidad de registros categoricos de manera aleatoria
def get_categorical_dataset_simulated(simulation_extended, category_cols, size):
    # la columna n = 1 servira para realizar la funcion de agrupacion count en el dataframe
    simulation_extended['n'] = 1

    # Generamos la agrupacion por columnas categoricas con su respectiva funcion de agregacion count para obtener la frecuencia con la que aparecen cada agrupacion
    grouped_combination = simulation_extended.groupby(
        category_cols,
        as_index=False
    ).agg({
        'n': 'count'
    })

    # Renombramos los nombres de las columnas, en este caso seran las mismas columnas categoricas + la ultima columna n que era la relacionada con la frecuencia o conteo
    grouped_combination.columns = category_cols + ['count']

    # Agregamos otra columna extra de prob para almacenar la probabilidad con la que aparecen cada grupo de columnas categoricas
    grouped_combination['prob'] = grouped_combination['count'] / dataframe_list[4].shape[0]

    # Generamos aleatoriamente un arreglo de indices en base a las combinaciones generadas anteriormente manteniendo sus probabilidades
    rand_index = np.random.choice(
        grouped_combination.index,
        size = size,
        p = list(grouped_combination['prob'])
    )

    # Retornamos los 100k registros aleatorios ya con sus respectivos valores, no como indices nada mas
    return grouped_combination.loc[rand_index, category_cols]   

# Generar los registros de manera aleatoria para las columnas numericas
def get_numeric_column_simulated( simulated , simulation_extended , category_cols , column_name ):
    # Funcion de agregacion para obtener el min, max, mean y std por cada agrupacion de columnas categoricas
    a1 = simulation_extended.groupby(
        category_cols
        , as_index = False
    ).agg(
        {
            column_name: ["min","max","mean","std"]
        }
    )

    a1.columns = [c for c in category_cols] + ["Min", "Max", "Mean", "Std"]  # Renombra las columnas de a1

    ColumnSimulated = pd.DataFrame() # Inicializa un nuevo dataframe vacio en este caso
    
    # Recorre los indices de a1
    for i in a1.index:
        # Toma el registro completo del indice i, el registro completo son toda la fila de valores de ese indice
        rs = a1.loc[i]

        # reasigna area_subarea y crea una copia de todos los registros de simulated (que al principio era una copia de area_subarea sin la columna n) que contengan el indice i
        area_subarea = simulated.loc[i].copy()    

        # Genera los valores aleatorios en base a su resumen estadistico de min, max, mean y std
        data = generate_truncated_normal_data(
            rs["Min"] - 1 if rs["Std"] == 0 else rs["Min"],
            rs["Max"] + 1 if rs["Std"] == 0 else rs["Max"],
            1 if rs["Std"] == 0 else rs["Std"],
            rs["Mean"], 
            area_subarea.shape[0]
        )

        # Agregamos la nueva columna a area_subarea
        area_subarea[column_name] = data

        # Concatenamos el conjunto de registros anteriores con el registro actual recorrido del ciclo
        ColumnSimulated = pd.concat( [ ColumnSimulated , area_subarea  ] )

    return ColumnSimulated.reset_index(drop=True) # Retornamos el resultado final restableciendo los indices
    

## Helpers

In [455]:
# Funcion encargada de calcular la columna con mayor cantidad de registros (Esto en caso de que el valor random sea False)
def calc_greater(cols_arr):
    max_values = 0 # Inicia en cero suponiendo que no hay nada todavia al inicio
    
    for col in cols_arr:
        col_type = col['type']

        if col_type == 'foreign':
            pass
        
        if col_type == 'date':
            max_values = max(max_values, pd.date_range(start=col['values']['min'], end=col['values']['max']).size)

        if col_type == 'category':
            max_values = max(max_values, len(col['values']))

    return max_values

# Funcion encargada de evaluar que en efecto las columnas relacionadas existan asi como sus posibles columnas
def validate_relationships(all_dfs, reference_df):
    # Primero valido que exista el dataframe
    df_name, df_column = reference_df.split('.')

    # Validar por nombre del dataframe
    if df_name not in all_dfs:
        raise ValueError(f'El dataframe {df_name} no existe :(')

    # Sacamos las columnas validas
    columns = list(all_dfs[df_name].columns)

    # Validamos por columna
    if df_column not in columns:
        raise ValueError(f'La columna {df_column} no existe :( en el dataframe {df_name}. Las columnas validas son: {columns}.')

# Funcion que retorna un arreglo con todas las dependencias de un objeto que contiene alguna configuracion
def extract_dependencies(obj):
    dependencies = []
    
    for col in obj['columns']:
        if col['type'] == 'foreign':
            df_name = col['values'].split('.')[0]
            dependencies.append(df_name)
    return dependencies

# Funcion encargada de reordenar por dependencias el arreglo conf_list que puede que su orden no sea el de acuerdo a sus dependencias
def reorder_by_dependencies(arr): # Usando el algoritmo de insertion sort
    for i, value in enumerate(arr[1:], start=1):
        value_dependencies = extract_dependencies(value)
        j = i - 1

        # Validar que j no sea menor que cero y que las dependencias del valor actual recorrido esten en el valor de la izq
        while j >= 0 and any(dep in [d['ds'] for d in arr[:j+1]] for dep in value_dependencies):
            arr[j + 1] = arr[j]
            j -= 1
        
        # Remplazar el valor actual despues del elemento del que dependa
        arr[j + 1] = value
    return list(reversed(arr))

all_dfs = {} # Diccionario que contiene todos los dataframes creados

# Funcion que genera un dataframe a partir de su configuracion
def generate_df(sett):
    result_df = pd.DataFrame()

    # Aqui mandamos a llamar a la funcion calc_greater para determinar cual columna contiene la mayor cantidad de registros
    max_values = calc_greater(sett['columns']) if sett['random'] == False else sett['random_rows']
    
    for col in sett['columns']:
        col_name, col_type = col['name'], col['type']
        
        if col_type == 'category':
            values = col['values']

            if (len(values) != max_values):
                result_df[col_name] = generate_category_random_values(values, max_values)
            else:    
                result_df[col_name] = values

        if col_type == 'date':
            min = col['values']['min']
            max = col['values']['max']
            date_range = pd.date_range(start=min, end=max)
            
            if (date_range.size != max_values):
                result_df[col_name] = generate_date_random_values(min, max, max_values)
            else:    
                result_df[col_name] = pd.Series(date_range)

        if col_type == 'unique':
            result_df[col_name] = generate_unique_random_values(max_values)

        if col_type == 'foreign':
            col_reference_name, col_reference_area = col['values'].split('.')
            
            validate_relationships(all_dfs, col['values']) # Si no se traba aqui por un error de validacion entonces continua su ejecucion

            result_df[col_name] = generate_foreign_random_values(all_dfs[col_reference_name][col_reference_area], max_values)

        if col_type == 'numeric':
            if 'std' in col['values'].keys() and 'mean' in col['values'].keys():
                result_df[col_name] = generate_numeric_random_values(col['values']['min'], col['values']['max'], col['values']['std'], col['values']['mean'], max_values)
            else:
                result_df[col_name] = generate_numeric_random_values(col['values']['min'], col['values']['max'], None, None, max_values)
            
    result_df = pd.DataFrame(result_df)
    result_df.name = sett['ds']

    all_dfs[result_df.name] = result_df
    return result_df

## Main

In [341]:
def build_dataframes(conf_list):
    temp_conf_list = reorder_by_dependencies(conf_list.copy())
    df_arr = []

    for df in temp_conf_list:
        df_arr.append(generate_df(df))

    # Reordenar considerando el orden original (conf_list)
    return  [j for i in conf_list for j in df_arr if i['ds'] == j.name]

## Settings

In [463]:
d1 = {
    "ds": "dataset",
    "columns": [
        {
            "name": "area",
            "type": "category",
            "values": ["TI", "FIN", "HR"]
        },
        {
            "name": "id",
            "type": "unique"
        },
        {
            "name": "Fecha",
            "type": "date",
            "values": {
                "min": "2024-01-01",
                "max": "2024-02-28"
            }
        }
    ],
    "random": False
}

# d2 = {
#     "ds": "dataset2",
#     "columns": [
#         {
#             "name": "id",
#             "type": "unique"
#         },
#         {
#             "name": "area",
#             "type": "foreign",
#             "values": "dataset.area"
#         },
#         {
#             "name": "subarea",
#             "type": "category",
#             "values": ["SA1", "SA2", "SA3", "SA4"]
#         }
#     ],
#     "random": False
# }

# d3 = {
#     "ds": "dataset3",
#     "columns": [
#         {
#             "name": "id",
#             "type": "unique"
#         }, 
#         {
#             "name": "subarea",
#             "type": "foreign",
#             "values": "dataset2.id"
#         },
#         {
#             "name": "income",
#             "type": "numeric",
#             "values": {
#                 "min": 0,
#                 "max": 10000
#             }
#         },
#         {
#             "name": "goal",
#             "type": "numeric",
#             "values": {
#                 "min": 0,
#                 "max": 50000,
#                 "std": 20000,
#                 "mean": 25000
#             }
#         }
#     ],
#     "random": True,
#     "random_rows": 1000
# }

# d4 = {
#     "ds": "dataset4",
#     "columns": [
#         {
#             "name": "area",
#             "type": "category",
#             "values": ["TIC", "FIN", "HR", "MKT"]
#         },
#         {
#             "name": "Fecha",
#             "type": "date",
#             "values": {
#                 "min": "2024-01-01",
#                 "max": "2024-02-28"
#             }
#         },
#         {
#             "name": "subarea",
#             "type": "foreign",
#             "values": "dataset3.id"
#         },
#         {
#             "name": "id",
#             "type": "unique"
#         }
#     ],
#     "random": False
# }

# d5 = {
#     "ds": "dataset5",
#     "columns": [
#         {
#             "name": "id",
#             "type": "unique"
#         }, 
#         {
#             "name": "area",
#             "type": "category",
#             "values": ["TIC", "FIN", "HR", "MKT"]
#         },
#         {
#             "name": "subarea",
#             "type": "category",
#             "values": ["SA1", "SA2", "SA3", "SA4"]
#         },
#         {
#             "name": "income",
#             "type": "numeric",
#             "values": {
#                 "min": 0,
#                 "max": 10000
#             }
#         },
#         {
#             "name": "goal",
#             "type": "numeric",
#             "values": {
#                 "min": 0,
#                 "max": 50000,
#                 "std": 20000,
#                 "mean": 25000
#             }
#         }
#     ],
#     "random": True,
#     "random_rows": 1000
# }


In [465]:
dataframe_list = build_dataframes([d1])
dataframe_list[0]

,area,id,Fecha
0,HR,8d98cbc793194018,2024-01-01
1,TI,3b9d599c8dc84974,2024-01-02
2,TI,141c2841b70346c7,2024-01-03
3,HR,79f7075b81534b27,2024-01-04
4,HR,bcb6a9378b2e4669,2024-01-05
5,TI,3f73aea4a2c343e6,2024-01-06
6,HR,ef7e5d0f90f64b4c,2024-01-07
7,FIN,6ab671f3c3424dfe,2024-01-08
8,TI,74cbf5b5bd9e4462,2024-01-09
9,FIN,e765c2a516a9455e,2024-01-10


In [411]:
conf_list = [d3, d2, d1, d4, d5]

dataframe_list = build_dataframes(conf_list)

In [413]:
dataframe_list[4]

,id,area,subarea,income,goal
0,8397eee60f4548bf,HR,SA4,5147.630,5135.483
1,4a00f760f4c24dbb,FIN,SA1,4735.727,16955.878
2,5170befff4fa4945,MKT,SA4,1828.712,20344.054
3,ccf5b77fb6384c23,MKT,SA3,8722.724,41442.864
4,e493b429f2524a8c,FIN,SA3,1203.229,40248.155
...,...,...,...,...,...
995,07e8a0c6a1174fbc,MKT,SA2,865.967,16617.463
996,d318fd0fbf9f4259,MKT,SA3,7167.053,40948.733
997,d970bd2ce37648aa,MKT,SA4,594.403,27781.911
998,5518a135bcd94d5f,TIC,SA1,6244.482,44809.949


## Requisito #5

In [416]:
# dataframe_list[4] en este caso es el dataframe con random = True
simulation_extended = dataframe_list[4]

In [418]:
simulation_extended

,id,area,subarea,income,goal
0,8397eee60f4548bf,HR,SA4,5147.630,5135.483
1,4a00f760f4c24dbb,FIN,SA1,4735.727,16955.878
2,5170befff4fa4945,MKT,SA4,1828.712,20344.054
3,ccf5b77fb6384c23,MKT,SA3,8722.724,41442.864
4,e493b429f2524a8c,FIN,SA3,1203.229,40248.155
...,...,...,...,...,...
995,07e8a0c6a1174fbc,MKT,SA2,865.967,16617.463
996,d318fd0fbf9f4259,MKT,SA3,7167.053,40948.733
997,d970bd2ce37648aa,MKT,SA4,594.403,27781.911
998,5518a135bcd94d5f,TIC,SA1,6244.482,44809.949


In [420]:
# Arreglo de columnas categoricas como numericas del dataframe al cual se le esta realizando el analisis
category_cols = ['area', 'subarea']
numeric_cols = ['income', 'goal']

## Analisis mejor combinacion de variables categoricas 

In [423]:
simulation_extended['n'] = 1
grouped_combination = simulation_extended.groupby(
    category_cols,
    as_index=False
).agg({
    'n': 'count'
})

grouped_combination.columns =  category_cols + ['count']
grouped_combination['prob'] = grouped_combination['count'] / dataframe_list[4].shape[0]

El resultado de este analisis se imprime mas abajo para realizar la comparacion con lo que pasa despues de generar los n cantidad de registros aleatorios

In [426]:
# area_subarea sera el resultado que devuelve la funcion get_categoricas_dataset_simulated
area_subarea = get_categorical_dataset_simulated(simulation_extended, category_cols, 100000)
area_subarea

,area,subarea
5,HR,SA2
2,FIN,SA3
9,MKT,SA2
9,MKT,SA2
12,TIC,SA1
...,...,...
1,FIN,SA2
9,MKT,SA2
11,MKT,SA4
1,FIN,SA2


## Analisis mejor combinacion de variables categoricas despues de generar los n cantidad de registros

In [429]:
area_subarea['n'] = 1
result = area_subarea.groupby(
    category_cols,
    as_index=False,
).agg({
    'n': 'count'
})

result.columns = category_cols + ['count']
result['prob'] = result['count'] / area_subarea.shape[0]

## Comparacion del analisis antes y despues de generar los n cantidad de registros

In [432]:
# Mejor combinacion de variables categoricas desde la primera agrupacion
grouped_combination.sort_values(by='count', ascending=False)

,area,subarea,count,prob
15,TIC,SA4,77,0.077
14,TIC,SA3,73,0.073
7,HR,SA4,70,0.070
11,MKT,SA4,70,0.070
12,TIC,SA1,68,0.068
13,TIC,SA2,66,0.066
3,FIN,SA4,64,0.064
2,FIN,SA3,60,0.060
9,MKT,SA2,60,0.060
1,FIN,SA2,59,0.059


In [434]:
# Probabilidades de las mejores combinaciones despues de haber generado los 100k registros
result.sort_values(by='count', ascending=False)

,area,subarea,count,prob
15,TIC,SA4,7754,0.07754
14,TIC,SA3,7273,0.07273
7,HR,SA4,7084,0.07084
11,MKT,SA4,7053,0.07053
12,TIC,SA1,6756,0.06756
13,TIC,SA2,6643,0.06643
3,FIN,SA4,6412,0.06412
9,MKT,SA2,5979,0.05979
1,FIN,SA2,5919,0.05919
2,FIN,SA3,5851,0.05851


Como se puede ver en el ejemplo anterior, las probabilidades despues de haber generado los 100k registros se mantiene "casi" igual como desde la agrupacion original.

In [448]:
# simulated en este caso es una copia de area_subarea excluyendo la variable n que no pertenece al dataframe original
# area_subarea contiene los 100k registros generados aleatoriamente
simulated = area_subarea.drop(columns='n', axis=1).copy()

# Restablece los indices a una secuencia por default antes: [0, 0, 0, ..., n, n] ahora: [0,1,2,3,...,n]
final_simulation = simulated.sort_index().reset_index(drop=True).copy()

# Recorremos el arreglo de columnas numericas
for nc in numeric_cols:
    # Por cada valor numerico, se generan los demas registros aleatoriamente
    # simulated = copia de los 100k registros generados aleatorios
    # simulation_extended = dataframe al cual se le esta realizando el analisis. En este caso este dataframe se creo desde el inicio con 1k registros
    # category_cols = arreglo con las columnas categoricas
    # nc = columna numerica actual a medida se recorre el ciclo de las columnas numericas
    dfn = get_numeric_column_simulated(simulated, simulation_extended, category_cols, nc) 

    # Combina el dataframe actual de final_simulation con el dataframe que se va generando a medida recorra el ciclo
    # Los combina por sus indices (Tomando en cuenta que los indices son unicos)
    final_simulation = pd.merge(
        final_simulation
        , dfn.loc[:,[nc]]
        , left_index=True
        , right_index=True
    )

In [450]:
final_simulation

,area,subarea,income,goal
0,FIN,SA1,3937.850,23942.143
1,FIN,SA1,7961.238,12521.008
2,FIN,SA1,6142.741,42560.677
3,FIN,SA1,2222.090,24755.064
4,FIN,SA1,2284.060,32804.851
...,...,...,...,...
99995,TIC,SA4,5996.355,34592.092
99996,TIC,SA4,7204.380,26231.485
99997,TIC,SA4,3587.887,17710.652
99998,TIC,SA4,9874.974,34668.247
